### In this notebook we understand the implementation of forward hooks to get output activations from intermediate layers

### 1. Import libraries

In [1]:
import torch
from torch import nn
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from collections import OrderedDict
import matplotlib.pyplot as plt

### 2. Create model

In [2]:
model = nn.Sequential(OrderedDict([('fc1',nn.Linear(128,64)),
                       ('sigmoid', nn.Sigmoid()),
                       ('fc2', nn.Linear(64,5)),
                       ('softmax',nn.Softmax(dim=1))]))
#define intput
x = torch.rand(1, 128)



### 3. Define hooks

In [12]:
activations = {} #this will store the outputs of layers

### This function returns the activations
def get_activation(name): #name -> layer name. As we cannot send name to activation_hook directly we create an abstraciton to send name
    def activation_hook(inst, inp, out):
        """Run activation hook.
        Parameters
        ----------
        inst : torch.nn.Module
            The layer we want to attach the hook to.
        inp : tuple of torch.Tensor
            The input to the `forward` method.
        out : torch.Tensor
            The output of the `forward` method.
        """
        activations[name] = out.data
    return activation_hook





#### 3.1 Method 1: Get layer by name

In [13]:
model

Sequential(
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=64, out_features=5, bias=True)
  (softmax): Softmax(dim=1)
)

In [14]:
#Register forward hooks for the layers required
model.fc1.register_forward_hook(get_activation('fc1'))
model.sigmoid.register_forward_hook(get_activation('sigmoid'))
model.fc2.register_forward_hook(get_activation('fc2'))
model.softmax.register_forward_hook(get_activation('softmax'))

y = model(x)

In [15]:
for name, outputs in activations.items():
    print(name,' : ', outputs.shape)
print('\nsoftmax output: ', activations['softmax'])

fc1  :  torch.Size([1, 64])
sigmoid  :  torch.Size([1, 64])
fc2  :  torch.Size([1, 5])
softmax  :  torch.Size([1, 5])
  :  torch.Size([1, 5])

softmax output:  tensor([[0.1325, 0.2931, 0.2387, 0.1624, 0.1733]])


In [16]:
print(y.data)

tensor([[0.1325, 0.2931, 0.2387, 0.1624, 0.1733]])


> We can see above that the forward hook for softmax returns the same values as output from the model

#### 3.2 Method 2: Get all layers by using model.named_modules()
We can get name and the layers using named_modules()

In [17]:
for name, module in model.named_modules():
    print(name, ':', module)

 : Sequential(
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (sigmoid): Sigmoid()
  (fc2): Linear(in_features=64, out_features=5, bias=True)
  (softmax): Softmax(dim=1)
)
fc1 : Linear(in_features=128, out_features=64, bias=True)
sigmoid : Sigmoid()
fc2 : Linear(in_features=64, out_features=5, bias=True)
softmax : Softmax(dim=1)


In [18]:
activations = {} #this will store the outputs of layers
#register forward hooks
for name, layer in model.named_modules():
    layer.register_forward_hook(get_activation(name))


y = model(x)

In [19]:
for name, outputs in activations.items():
    print(name,' : ', outputs.shape)
print('\nsoftmax output: ', activations['softmax'])

fc1  :  torch.Size([1, 64])
sigmoid  :  torch.Size([1, 64])
fc2  :  torch.Size([1, 5])
softmax  :  torch.Size([1, 5])
  :  torch.Size([1, 5])

softmax output:  tensor([[0.1325, 0.2931, 0.2387, 0.1624, 0.1733]])


In [20]:
activations

{'fc1': tensor([[ 0.4593,  0.1147,  0.2386,  0.2440, -0.0586, -0.1439, -0.2947, -0.1832,
           0.0562, -0.3983, -0.3785, -0.2134,  0.3738,  0.5855,  0.1553,  0.4979,
          -0.3141,  0.1874, -0.1940,  0.1261,  0.0372,  0.0670, -0.3955,  0.1895,
           0.4422, -0.0744, -0.0892,  0.4032,  0.1033, -0.2076,  0.1923,  0.2056,
          -0.5855,  0.1144,  0.1287,  0.5942, -0.4269,  0.2187,  0.0882,  0.3308,
           0.4109, -0.2804, -0.2610, -0.0395,  0.6415,  0.5011, -0.2736, -0.1263,
          -0.3721, -0.2963,  0.2845,  0.0195,  0.1062,  0.1703, -0.3771, -0.1242,
           0.0838,  0.3291,  0.1005, -0.3307, -0.1236,  0.1868,  0.3189,  0.6751]]),
 'sigmoid': tensor([[0.6128, 0.5287, 0.5594, 0.5607, 0.4854, 0.4641, 0.4268, 0.4543, 0.5140,
          0.4017, 0.4065, 0.4469, 0.5924, 0.6423, 0.5387, 0.6220, 0.4221, 0.5467,
          0.4516, 0.5315, 0.5093, 0.5167, 0.4024, 0.5472, 0.6088, 0.4814, 0.4777,
          0.5995, 0.5258, 0.4483, 0.5479, 0.5512, 0.3577, 0.5286, 0.5321, 0.6

In [23]:
y.data

tensor([[0.1325, 0.2931, 0.2387, 0.1624, 0.1733]])